<a href="https://colab.research.google.com/github/zoedesimone/DiffusionWorldViewer/blob/main/DiffusionWorldViewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DiffusionWorldViewer
This Colab contains code for the paper What is a Fair Diffusion Model? Designing Generative Text-To-Image Models to Incorporate Various Worldviews.

Authors: Zoe De Simone, Angie Boggust, Arvind Satyanarayan, Ashia Wilson

A demo of the DiffusionWorldViewer can be run in Google Colab, by running this notebook.

## Citation
If you find the Embedding Comparator useful in your work, please cite:



```
@article{desimone2023DiffusionWorldViewer,
      title={What is a Fair Diffusion Model? Designing Generative Text-To-Image Models to Incorporate Various Worldviews},
      author={ Zoe De Simone and Angie Boggust and Arvind Satyanarayan and Ashia Wilson},
      year={2023},
      journal={}
}

```


## Tips for Running this Notebook

Before you begin, ensure the notebook is using Hardware accelerator: GPU.

You can change this in Runtime > Change Runtime Type

Runtime > Run All to run all of the cells and generate the UI. Next, scroll to the bottom of the notebook and click on the generated https://[numbers].gradio.live link to open the DiffusionWorldViewer in your browser.


## Attributions
This notebook builds on code from [FairDiffusion](https://github.com/ml-research/Fair-Diffusion) and the [FairFace classifier](https://github.com/joojs/fairface).

# 0.0 | Imports 📦

In [1]:
# @title Import Libraries, Models, etc...

!pip install accelerate  # this is to reduce CPU model load overhead
!nvidia-smi

# Install dependencies for SEGA
!pip install git+https://github.com/ml-research/semantic-image-editing &> /dev/null
!pip install ftfy &> /dev/null

#SEGA Imports
#utils
from matplotlib import pyplot as plt
import torch
from PIL import Image

# SEGA
from semdiffusers import SemanticEditPipeline

# Imports for FairFace
import math
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf

import random
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import torch.nn as nn
from torchvision import models, transforms
import dlib
import os
from tqdm import tqdm
from PIL import Image
import glob

import shutil
import os

!pip install dlib

# Download FairFace dataset
!pip install --upgrade --no-cache-dir gdown > /dev/null
!gdown https://drive.google.com/uc?id=1J7EJl9Y-JbAo0SUf5B3AYLIiHrD75M-j
!gdown  https://drive.google.com/uc?id=1H11xsJDcxDNxkT2yzSqZlCWh0IRMgAkT

# Clone FairFace repo and move files
! git clone  https://github.com/dchen236/FairFace
!mkdir 'dlib_models'
!mv "res34_fair_align_multi_4_20190809.pt" "dlib_models/res34_fair_align_multi_4_20190809.pt"
!mv "res34_fair_align_multi_7_20190809.pt" "dlib_models/res34_fair_align_multi_7_20190809.pt"# @title Clone FairFace repo and move files
! git clone  https://github.com/dchen236/FairFace
!mkdir 'dlib_models'
!mv "res34_fair_align_multi_4_20190809.pt" "dlib_models/res34_fair_align_multi_4_20190809.pt"
!mv "res34_fair_align_multi_7_20190809.pt" "dlib_models/res34_fair_align_multi_7_20190809.pt"

# Download Stable Diffusion Model
model_id = 'stabilityai/stable-diffusion-2-base'
device = 'cuda'

# title Visualization Imports
import plotly.express as px
#from ipywidgets import interact, widgets

!pip install tqdm
import tqdm

#Gradio Installs
!pip install gradio

import gradio as gr
from gradio.inputs import Slider
import time

!pip install -q gradio altair
import altair as alt

from PIL import Image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.8 MB/s eta 0:00:00
Mon Sep 18 16:25:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-----

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/semdiffusers/pipeline_latent_edit_diffusion.py:12: FutureWarning: Importing `DiffusionPipeline` or `ImagePipelineOutput` from diffusers.pipeline_utils is deprecated. Please import from diffusers.pipelines.pipeline_utils instead.
  from diffusers.pipeline_utils import DiffusionPipeline


Downloading...
From: https://drive.google.com/uc?id=1J7EJl9Y-JbAo0SUf5B3AYLIiHrD75M-j
To: /content/res34_fair_align_multi_7_20190809.pt
100% 85.3M/85.3M [00:00<00:00, 148MB/s]
Downloading...
From: https://drive.google.com/uc?id=1H11xsJDcxDNxkT2yzSqZlCWh0IRMgAkT
To: /content/res34_fair_align_multi_4_20190809.pt
100% 85.3M/85.3M [00:00<00:00, 206MB/s]
Cloning into 'FairFace'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 220 (delta 4), reused 6 (delta 3), pack-reused 211
Receiving objects: 100% (220/220), 14.23 MiB | 1.42 MiB/s, done.
Resolving deltas: 100% (111/111), done.
mv: target 'files' is not a directory
fatal: destination path 'FairFace' already exists and is not an empty directory.
mkdir: cannot create directory ‘dlib_models’: File exists
mv: cannot stat 'res34_fair_align_multi_4_20190809.pt': No such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 75

In [3]:
#@title ##Import helper functions from github

!git clone https://github.com/zoedesimone/DiffusionWorldViewer.git

#move files from
source_folder = '/content/DiffusionWorldViewer'
destination_folder = '/content'

# List all files in the source folder
files = os.listdir(source_folder)

# Move each file to the destination folder
for file in files:
    source_path = os.path.join(source_folder, file)
    destination_path = os.path.join(destination_folder, file)
    shutil.move(source_path, destination_path)

# Remove the empty source folder after moving all files
if not os.listdir(source_folder):
    os.rmdir(source_folder)


from util import *

fatal: destination path 'DiffusionWorldViewer' already exists and is not an empty directory.


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
#@title #Make Image Folders
!mkdir 'baseline_images/'
#!mkdir 'detected_faces'
!mkdir "edited_images/"
!mkdir 'results/'

# Import the pretrained Stable Diffusion Model
pipe_edit =  SemanticEditPipeline.from_pretrained(model_id,safety_checker=None,)
pipe_edit = pipe_edit.to(device)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
#@title #High Level Editing Functions

def generate_edited_images(prompt, radio_btn,rel_slider, gender_checked, race_checked, age_checked, progress=gr.Progress(track_tqdm=True)):
    """
    Generate edited images based on user-defined parameters.

    Args:
        prompt (str): The text prompt used for image generation.
        radio_btn (str): The selected radio button for image generation type.
        rel_slider (float): The slider value for relative image generation.
        gender_checked (list): List of selected gender options.
        race_checked (list): List of selected race options.
        age_checked (list): List of selected age options.
        progress (gr.Progress, optional): Progress bar for tracking image generation. Defaults to gr.Progress(track_tqdm=True).

    Returns:
        str: A message indicating the completion of image generation.
    """

    n_images = 5
    new_image_info_list = []

    if (radio_btn == "Absolute cat."):
      for _ in range(n_images):
          new_image_info = [
              gender_checked[random.randint(0, len(gender_checked) - 1)] + " person",
              race_checked[random.randint(0, len(race_checked) - 1)] + " person",
              age_checked[random.randint(0, len(age_checked) - 1)] + " years old"
          ]
          new_image_info_list.append(new_image_info)

    elif (radio_btn == "Relative to the baseline"):
        df_g, df_r, df_a =  generate_baseline_demographics()

        df_g = interpolate_relative(df_g, rel_slider)
        df_r = interpolate_relative(df_r, rel_slider)
        df_a = interpolate_relative(df_a, rel_slider)

        # Select values based on weights
        print(df_g["value"])
        #value has to be in the range of 0-1 for random choice
        selected_g = np.random.choice(df_g["variable"], size=n_images, p=df_g["value"])
        selected_r = np.random.choice(df_r["variable"], size=n_images, p=df_r["value"])
        selected_a = np.random.choice(df_a["variable"], size=n_images, p=df_a["value"])

        #ater random choice multiply value by 100
        df_g["value"] = (df_g["value"] * 100).round(2)
        df_a["value"] = (df_a["value"] * 100).round(2)
        df_r["value"] = (df_r["value"] * 100).round(2)

        g = selected_g.tolist()
        r = selected_r.tolist()
        a = selected_a.tolist()

        new_image_info_list  =[]

        for i in range(n_images):
          new_image_info = [
                      g[i],
                      r[i],
                      a[i] + " years old" ]
          new_image_info_list.append(new_image_info)

    elif radio_btn == "Parity":
      #all the same
        data_g = {
                  "variable": ["Female", "Male"],
                  "value": [0.5, 0.5],
          }

        data_r = {
                    "variable": ["Black", "East Asian", "Hispanic", "Indian", "Middle Eastern", "Southeast Asian", "White"],
                    "value": [0.143, 0.143, 0.143, 0.143, 0.143, 0.143, 0.142],
                }

        data_a = {
                    "variable": ["0-2", "03-09", "10-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70+"],
                    "value": [0.11111111111, 0.11111111111, 0.11111111111, 0.11111111111, 0.11111111111, 0.11111111111, 0.11111111111, 0.11111111111, 0.11111111111],
                }

        # Select values based on weights
        selected_g = np.random.choice(data_g["variable"], size=n_images, p=data_g["value"])
        selected_r = np.random.choice(data_r["variable"], size=n_images, p=data_r["value"])
        selected_a = np.random.choice(data_a["variable"], size=n_images, p=data_a["value"])


        g = selected_g.tolist()
        r = selected_r.tolist()
        a = selected_a.tolist()

        new_image_info_list  =[]

        for i in range(n_images):
          new_image_info = [
                      g[i] + " person",
                      r[i]+ " person",
                      a[i] + " years old" ]
          new_image_info_list.append(new_image_info)

        """
        #all the same
        gender_checked = ["Female", "Male"],
        race_checked = ["Black", "East Asian", "Hispanic", "Indian","Middle Eastern", "Southeast Asian", "White"]
        age_checked = ["0-2","03-09","10-19", "20-29","30-39","40-49","50-59","60-69","70+"]

        for _ in range(n_images):

            new_image_info = [
                gender_checked[random.randint(0, len(gender_checked) - 1)]+ " person",
                race_checked[random.randint(0, len(race_checked) - 1)]+ " person",
                age_checked[random.randint(0, len(age_checked) - 1)] + " years old"
            ]
            new_image_info_list.append(new_image_info)
        """


    elif radio_btn == "US demogr.":
        #generate edits based on %
        data_g = {
      "variable": ["Female", "Male"],
      "value": [0.51, 0.49],
          }

        data_r = {
                    "variable": ["Black", "East Asian", "Hispanic", "Indian", "Middle Eastern", "Southeast Asian", "White"],
                    "value": [0.13, 0.03, 0.19, 0.03, 0.03, 0.03, 0.56],
                }

        data_a = {
                    "variable": ["0-2", "03-09", "10-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70+"],
                    "value": [0.0339, 0.074, 0.1356, 0.1350, 0.1363, 0.129, 0.1273, 0.1096, 0.1193],
                }

        # Select values based on weights
        selected_g = np.random.choice(data_g["variable"], size=n_images, p=data_g["value"])
        selected_r = np.random.choice(data_r["variable"], size=n_images, p=data_r["value"])
        selected_a = np.random.choice(data_a["variable"], size=n_images, p=data_a["value"])


        g = selected_g.tolist()
        r = selected_r.tolist()
        a = selected_a.tolist()

        new_image_info_list  =[]

        for i in range(n_images):
          new_image_info = [
                      g[i] + " person",
                      r[i]+ " person",
                      a[i] + " years old" ]
          new_image_info_list.append(new_image_info)


    output_folder= "edited_images/"
    df_file_path="edited_images.csv"

    #clear folder of previous images if any
    # Iterate through items and delete files
    items = os.listdir(output_folder)
    for item in items:
        item_path = os.path.join(output_folder, item)
        if os.path.isfile(item_path):
            os.remove(item_path)
            print(f"Deleted file: {item_path}")

    images = []
    seed = 0
    # Print the generated image list
    for edit in progress.tqdm(new_image_info_list, desc="Generating images"):

        # perform the desired modifications
        edited_image = edit_image(prompt, edit, seed)
        save_image(edited_image, seed, prompt, output_folder, df_file_path, edited = True)
        images.append(edited_image)
        print(f"saved edited image with edit: {str(edit)}.")
        seed += 1
    return ""

def generate_sd_images(prompt, progress=gr.Progress(track_tqdm=True)):
    """
    Generates and saves StableDiffusion-generated images in batches of 1 to a specified `image_folder` and saves information to `csv_filepath`.

    The function runs StableDiffusion in batches to avoid running out of memory (CUDA out of MEMORY error).

    Args:
        prompt (str): The text prompt used for image generation.
        progress (gr.Progress, optional): Progress bar for tracking image generation. Defaults to gr.Progress(track_tqdm=True).

    Returns:
        str: A message indicating the completion of image generation.
    """
    seed = 0
    n_images = 5

    image_folder = "baseline_images/"
    df_file_path = "image_dataframe.csv"

    #clear folder of previous images if any
    # Iterate through items and delete files
    items = os.listdir(image_folder)
    for item in items:
        item_path = os.path.join(image_folder, item)
        if os.path.isfile(item_path):
            os.remove(item_path)
            print(f"Deleted file: {item_path}")

    seeds = np.arange(1, n_images, 1).tolist()  # save the seeds for later

    images = []
    lst = [0,1,2,3,4]

    for seed in progress.tqdm(lst, desc="Generating images"):
        print(f"Seed: {seed}")
        gen = torch.Generator(device=device).manual_seed(seed)
        org = pipe_edit(prompt=prompt, generator=gen, num_images_per_prompt=1, guidance_scale=7)
        save_image(org.images[0], seed, prompt, folder=image_folder, df_file_path=df_file_path, edited=False)
        images.append(org.images[0])

    return ""


def load_imgs_baseline(progress=gr.Progress()):
    """
    Load baseline images from the 'baseline_images' folder.

    Args:
        progress (gr.Progress, optional): Progress bar for tracking image loading. Defaults to gr.Progress().

    Returns:
        list: A list of loaded baseline images.
    """
    folder_path ="baseline_images/"
    images = os.listdir(folder_path)

    generated_imgs = []
     # Sort the image names
    sorted_images = sorted(images)

    generated_imgs = []
    for image_name in sorted_images:
        if image_name.endswith(".jpg") or image_name.endswith(".png"):
            image_path = os.path.join(folder_path, image_name)
            image = Image.open(image_path)
            generated_imgs.append(image)

    return generated_imgs

##Logos for header

In [6]:
# download mit logo
!wget -O mit_logo.svg https://www.google.com/url?sa=i&url=https%3A%2F%2Fcommons.wikimedia.org%2Fwiki%2FFile%3AMIT_logo.svg&psig=AOvVaw3lPOC0FFbT2ltZ2HOZ_ctc&ust=1692741290464000&source=images&cd=vfe&opi=89978449&ved=0CA8QjRxqFwoTCKiO7-ve7oADFQAAAAAdAAAAABAE
!wget -O csail_logo.png https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.csail.mit.edu%2Fnews%2Fmits-computer-science-and-artificial-intelligence-laboratory-csail-unveils-new-visual-identity&psig=AOvVaw07CO3n230prDa-SmRU3PsH&ust=1692741519058000&source=images&cd=vfe&opi=89978449&ved=0CA8QjRxqFwoTCJik0dnf7oADFQAAAAAdAAAAABAE

--2023-09-18 16:28:10--  https://www.google.com/url?sa=i
Resolving www.google.com (www.google.com)... 142.251.175.147, 142.251.175.105, 142.251.175.99, ...
Connecting to www.google.com (www.google.com)|142.251.175.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘mit_logo.svg’

mit_logo.svg            [ <=>                ]   1.37K  --.-KB/s    in 0s      

2023-09-18 16:28:10 (22.9 MB/s) - ‘mit_logo.svg’ saved [1402]

--2023-09-18 16:28:10--  https://www.google.com/url?sa=i
Resolving www.google.com (www.google.com)... 142.251.175.147, 142.251.175.105, 142.251.175.99, ...
Connecting to www.google.com (www.google.com)|142.251.175.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘csail_logo.png’

csail_logo.png          [ <=>                ]   1.37K  --.-KB/s    in 0s      

2023-09-18 16:28:10 (38.3 MB/s) - ‘csail_logo.png’ saved [1402]



In [7]:
import io

from PIL import Image  # https://pillow.readthedocs.io/en/4.3.x/
import requests  # http://docs.python-requests.org/en/master/


# example image url: https://m.media-amazon.com/images/S/aplus-media/vc/6a9569ab-cb8e-46d9-8aea-a7022e58c74a.jpg
def download_image(url, image_file_path):
    r = requests.get(url, timeout=4.0)
    if r.status_code != requests.codes.ok:
        assert False, 'Status code error: {}.'.format(r.status_code)

    with Image.open(io.BytesIO(r.content)) as im:
        im.save(image_file_path)

    print('Image downloaded from url: {} and saved to: {}.'.format(url, image_file_path))

In [8]:
csail_url = "https://www.csail.mit.edu/sites/default/files/2022-02/Screen%20Shot%202022-02-28%20at%204.01.44%20PM.png"
mit_url = "https://logowik.com/content/uploads/images/massachusetts-institute-of-technology-mit2419.jpg"
lids_url = "https://lids.mit.edu/sites/default/files/lids-logo.png"
download_image(csail_url, "csail_logo.png")
download_image(mit_url, "mit_logo.jpg")
download_image(lids_url, "lids_logo.png")

Image downloaded from url: https://www.csail.mit.edu/sites/default/files/2022-02/Screen%20Shot%202022-02-28%20at%204.01.44%20PM.png and saved to: csail_logo.png.
Image downloaded from url: https://logowik.com/content/uploads/images/massachusetts-institute-of-technology-mit2419.jpg and saved to: mit_logo.jpg.
Image downloaded from url: https://lids.mit.edu/sites/default/files/lids-logo.png and saved to: lids_logo.png.


In [9]:
# Open and load your images
image1 = Image.open("mit_logo.jpg")
image2 = Image.open("csail_logo.png")
image3 = Image.open("lids_logo.png")

# ... Load other images
# Define the target x-size for scaling
target_x_height = 300

# Scale down images while maintaining aspect ratio
new_size1 = (target_x_height, int(image1.height * target_x_height / image1.width))
scaled_image1 = image1.resize(new_size1)

new_size2 = (target_x_height, int(image2.height * target_x_height / image2.width))
scaled_image2 = image2.resize(new_size2)

new_size3 = (target_x_height, int(image3.height * target_x_height / image3.width))
scaled_image3 = image3.resize(new_size2)

# ... Scale other images
# Determine the total height for the combined image
total_height = max(scaled_image1.height, scaled_image2.height, scaled_image3.height)

# Create a new blank canvas with a white background
background_color = (255, 255, 255)  # White color in RGB
combined_image = Image.new('RGB', (target_x_height * 3, total_height), background_color)

# Paste scaled images onto the canvas
combined_image.paste(scaled_image1, (0, (total_height - scaled_image1.height) // 2))
combined_image.paste(scaled_image2, (target_x_height, (total_height - scaled_image2.height) // 2))
combined_image.paste(scaled_image3, (target_x_height * 2, (total_height - scaled_image3.height) // 2))


# Save the combined image as a JPG
combined_image.save('logos.jpg', format='JPEG')

# 1.0 UI blocks

In [10]:
#@title Image Galleries

#Fix height of gallery to avoid wierd resizing as images are loaded
css = '''
.generating {
    border: none !important;
}

#gallery {
  border: 0px;
  margin: 0px;
  height: 100px;
  width: auto;
}

'''
#Right Gallery
with gr.Blocks(theme=gr.themes.Default(spacing_size=gr.themes.sizes.spacing_sm), css = css).queue() as right_gallery:

  gallery_img_right = gr.Gallery(
              label="Generated images", show_label=False, elem_id="gallery", container = False
          ).style(columns=[5], rows=[1], object_fit= "contain", height="50")

  right_gallery.load(fn=load_imgs, inputs= None, outputs=gallery_img_right,
                show_progress=True, every=10)

right_gallery.launch(debug = False)

#Left Gallery
with gr.Blocks(theme=gr.themes.Default(spacing_size=gr.themes.sizes.spacing_sm), css = css).queue() as left_gallery:

  gallery_img_left = gr.Gallery(
              label="Generated images", show_label=False, elem_id="gallery",container = False
          ).style(columns=[5], rows=[1], object_fit= "contain", height="50")

  left_gallery.load(fn=load_imgs_baseline, inputs= None, outputs=gallery_img_left,
                show_progress=True, every=10)

left_gallery.launch(debug = False)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://18699fc942e085d373.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3d8697f7452b090f05.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [11]:
#@title ## Demographic Plots

css = '''
.generating {
    border: none !important;
}
#bar-plot {
  border: 5px solid pink;
  height: 50px;
  width: auto;
}
'''

with gr.Blocks(theme=gr.themes.Default(spacing_size=gr.themes.sizes.spacing_sm), css = css).queue() as left_plots:
  with gr.Column():

      gr.Markdown("""#"""), gr.Markdown("""#"""), gr.Markdown("""#"""),gr.Markdown("""#"""),
      gr.Markdown("""#"""), gr.Markdown("""#"""), gr.Markdown("""#"""),gr.Markdown("""#"""),
      gr.Markdown("""#"""), gr.Markdown("""#"""), gr.Markdown("""#""")

      base_plt_g = gr.Plot(label = "Gender predictions for generated images ", show_label = True, container = False, elem_id="bar-plot")
      gr.Markdown("""#"""), gr.Markdown("""#"""), gr.Markdown("""#"""),gr.Markdown("""#"""),
      gr.Markdown("""#"""), gr.Markdown("""#"""),

      base_plt_r = gr.Plot(label = "Race predictions for generated images", show_label = True, container = False, elem_id="bar-plot")
      gr.Markdown("""#"""), gr.Markdown("""#"""), gr.Markdown("""#"""),gr.Markdown("""#"""),
      gr.Markdown("""#""")

      base_plt_a = gr.Plot(label = "Age predictions for generated images", show_label = True, container = False, elem_id="bar-plot")
      left_plots.load(fn=plot_baseline_demographics, inputs= None, outputs=[base_plt_g, base_plt_r, base_plt_a],
                show_progress=False, every=10)

left_plots.launch(debug = False)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8dc9608b7592661f09.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# 2.0 DiffusionWorldViewer UI

In [13]:
#@title # Demo


css = '''
.generating {
    border: none !important;
}

#bar-plot {
  height: 50px;
  width: auto;
}
#logo{
  height: 50px;
  width: 50px;
  float:right;
}
footer {visibility: hidden}
}

#progress-bar{
  font-size: 8px;
  border: solid white 20px;
}

'''
logos = Image.open("logos.jpg")

with gr.Blocks(theme=gr.themes.Default(spacing_size=gr.themes.sizes.spacing_sm), css = css).queue() as demo:

  with gr.Row() as header:
    gr.Markdown("""# DiffusionWorldViewer """)
    gr.Markdown("""# """)
    gr.Image(type="pil", value= logos, container = False, elem_id="logo", show_download_button = False)

  prompt = gr.Textbox(lines=1, label="Input Prompt", show_label = False, placeholder="Enter your prompt. Ex. A photo of an accountant.", container=False)
  #btn_show_right = gr.Radio(["On", "Off"], label = "Editor", value = "Off")

  with gr.Row() as ui_sides:

    with gr.Column(scale = 4) as left_side:
      gr.Markdown("""## Stable Diffusion""")
      btn_imgs_left = gr.Button("Generate Stable Diffusion images 🎨🖌️").style(full_width=False)
      left_plots.render()
      left_gallery.render()
      progress_textbox_baseline = gr.Textbox(elem_id = "progress-bar", container= False, lines = 2)

    with gr.Column(scale = 4, visible = True) as right_side:
      gr.Markdown("""## Demographic worldview editor""")#Here you can specify the demographic distributions you wish to represent.
      btn_imgs_right = gr.Button("Generate edited images 🎨🖌️").style(full_width=False)

      with gr.Row() as btns:
        btn = gr.Radio(["Parity", "US demogr.", "Absolute cat.", "Relative to the baseline"], value = "Parity", label="Select which type of worldview to see represented in your images.", scale = 5, container=True, elem_id="radio-btn")
        rel_slider = gr.Slider(minimum = 0, maximum = 1, label = "Proximity to parity", scale = 1, interactive = False, container=True) #How close to demographic parity do you want to be?
      gender_check = gr.CheckboxGroup(["Male", "Female"], label = "Select what genders to represent.")
      plt_g = gr.Plot(label = "Gender predictions for generated images ", show_label = True, container = False, elem_id="bar-plot")

      race_check = gr.CheckboxGroup(["Black", "East Asian", "Hispanic", "Indian","Middle Eastern", "Southeast Asian", "White"], label = "Select what ethnicities to represent.")
      plt_r = gr.Plot(label = "Race predictions for generated images", show_label = True, container = False, elem_id="bar-plot")

      age_check = gr.CheckboxGroup(["0-2","03-09","10-19", "20-29","30-39","40-49","50-59","60-69","70+"], label ="Select what age groups to represent.")
      plt_a = gr.Plot(label = "Age predictions for generated images", show_label = True, container = False, elem_id="bar-plot")

      right_gallery.render()
      progress_textbox = gr.Textbox(elem_id = "progress-bar", container= False, lines = 2)

  def select_radio(radio_choice):
        """Update interface based on radio button selection."""
        if (radio_choice == "Absolute cat."):
            return {
                gender_check: gr.update(interactive=True),
                race_check: gr.update(interactive=True),
                age_check: gr.update(interactive=True)
            }
        elif (radio_choice == "Parity") or (radio_choice == "US demogr.") or (radio_choice == "Relative to the baseline"):
          return{
                gender_check: gr.update(interactive=False),
                race_check: gr.update(interactive=False),
                age_check: gr.update(interactive=False),
          }
  #Show the relative range based on relative radio button selection
  def rel_select(radio_choice):
        """Update interface based on radio button selection."""
        if (radio_choice =="Relative to the baseline"):
            return {
                rel_slider: gr.update(interactive=True),
            }
        return{
              rel_slider: gr.update(interactive=False),
        }

  def show_editing_ui(radio_choice):
    """Update interface based on radio button selection."""
    if(radio_choice == "On"):
      return {
                right_side: gr.update(visible = True)
            }
    return {
                right_side: gr.update(visible = False)
            }


  btn.change(
      rel_select,
      [ btn],
      rel_slider)

  btn.change(
      make_plots,
      [ btn, rel_slider, gender_check, race_check, age_check],
      [plt_g,plt_r, plt_a])

  #freeze radio buttons if they belong to USdemo or parity
  btn.change(
        select_radio,
        [btn],
        [gender_check, race_check, age_check])

  #update plots depending on demographic selections
  gender_check.change(
        make_plots,
        [btn, rel_slider, gender_check, race_check, age_check],
        [plt_g,plt_r, plt_a]
        )
  race_check.change(
        make_plots,
        [btn, rel_slider, gender_check, race_check, age_check],
        [plt_g,plt_r, plt_a]
        )
  age_check.change(
        make_plots,
        [btn, rel_slider, gender_check, race_check, age_check],
        [plt_g,plt_r, plt_a]
        )
  rel_slider.change(
        make_plots,
        [btn, rel_slider, gender_check, race_check, age_check],
        [plt_g,plt_r, plt_a]
        )

  # generate images for the gallery
  btn_imgs_left.click(generate_sd_images , inputs = [prompt], outputs= progress_textbox_baseline, queue = True)
  btn_imgs_right.click(generate_edited_images, inputs = [prompt, btn, rel_slider, gender_check, race_check, age_check], outputs= progress_textbox, queue = True)

  # Launch the interface
demo.launch(share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c6a74542a9e1a72f05.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
